<a href="https://colab.research.google.com/github/klajosw/python/blob/master/kl_python_sqlite_minta_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"> 
    <img src="https://raw.githubusercontent.com/klajosw/python/master/kl_mie_python_logo_250.jpg" 
         align="left" width="251" height="251">
    
</p>


<p> </p>



# Python alapok SQL db használat (Sqlite)

<https://klajosw.blogspot.com/>


---

# SqLite használata

## Kapcsolódás az adatbázishoz

In [0]:
## Modulok impontálása
import sqlite3
import os
import hashlib

## Belső függvények létrehozása

In [0]:
## Belső függveny eljárások definiálása
def collate_reverse(string1, string2):
    if string1 == string2:
        return 0
    elif string1 < string2:
        return 1
    else:
        return -1

## hashlib.md5 / hashlib.sha224 /// Titkosítás GDPR!!!
def md5sum(t):
    if type(t) == int:
        t = str(t)
    t = t.encode()    
    return hashlib.md5(t).hexdigest()  
    
def sha224sum(t):
    if type(t) == int:
        t = str(t)
    t = t.encode()    
    return hashlib.sha224(t).hexdigest()   

## Memoria adatbázis, tábla használati módszer

In [0]:
## Memoria adatbázis / tábla használata
sqlite3.enable_shared_cache(True) ### 

con = sqlite3.connect(":memory:")
con.isolation_level = None

con.create_collation("reverse", collate_reverse)  ## fuggvény használat

con.create_function("md5", 1, md5sum)

cur = con.cursor()

cur.execute("select md5(?)", ("foo",))
print(cur.fetchone()[0])

cur.execute("create table test(x)")
cur.executemany("insert into test(x) values (?)", [("a",), ("b",), ("d",) , ("c",)])
## cur.execute("select x from test order by x collate reverse")

cur.execute("select x from test order by x asc")

print('---- Kiíratás 0.  ----\n')
for row in cur:
    print(row)
con.close()

acbd18db4cc2f85cedef654fccc4a4d8
---- Kiíratás 0.  ----

('a',)
('b',)
('c',)
('d',)


## SQLite állomány adatbázis használata

In [0]:
## Állomány adatbázis
DB_FILE = "/content/sample_data/mydb.db"  # adatbázis neve
newdb=0    ## Új adatbázis akkor 1 legyen késöbb

if os.path.exists(DB_FILE ):  ## adatbázis állomány már létezik akkor törlésse
#   os.remove(DB_FILE)        ## törlésd végrehajtása
#   newdb = 1
   print('Törölve')


con = sqlite3.connect(DB_FILE)  ## ha nem létezik létrehozza (de ürsen)


def md5sum(t):  
    if type(t) == int:
        t = str(t)
    t = t.encode()    
    return hashlib.md5(t).hexdigest()  

con.create_function("md5", 1, md5sum)  ## beső fögvény használata/meghívása

cur = con.cursor()

if newdb == 1 :  ## Ha új adatbázis akkor a táblát létrekell hozni
    cur.execute(""" create table people (
      name_last  varchar(20), 
      age        integer)  """)
 
## Rekordok beszúrása / INSERT közvetlen megadással
    cur.execute("insert into people (name_last, age) values ('Lajos',   57)")    
    cur.execute("insert into people (name_last, age) values ('Béla',     53)")   
    cur.execute("insert into people (name_last, age) values ('Ildikó',     54)")  




    newPeople = (
    ('Dani' , 22),
    ('Tomi' , 21)
    )

    for person in newPeople:
        cur.execute("insert into people (name_last, age) values (?, ?)", person)

con.commit()  ## Beírás véglegesítése

##-----------------

print('---- Kiíratás 11  ----\n')
cur.execute("select * from people ")
for row in cur:
    for mezo in row:
        print(mezo)
        
print('---- Kiíratás 2.  ----\n')        
cur.execute("select * from people ")
for row in cur:        
        print('%s is %d years old.' % (row[0], row[1]))
##
print("Most törlök ", cur.execute("delete from people where name_last = 'Tomi' ").rowcount, "rekordot/sort.") 

print('---- Kiíratás 3.  ----\n')
cur.execute(" select name_last, age,  md5(name_last) as name_md5 from people ")

print("with column names:", cur.fetchone()[0])
print("with column names:", cur.fetchone())

print(cur.fetchall())  ## Összes további kiíratása


print('---- Kiíratás 4.  ----\n')
FIELD_MAX_WIDTH = 20  ## mezők maximális hossz beállítása
cur.execute(" select name_last, age,  md5(name_last) as name_md5 from people ")

# Fejléc
for fieldDesc in cur.description:
    print(fieldDesc[0].ljust(FIELD_MAX_WIDTH), end=' ')
print()          # Fejléc vége  
print('-' * 78)  # elválasztó sor

# For each row, print the value of each field left-justified within
# the maximum possible width of that field.
fieldIndices = range(len(cur.description))
for row in cur:
    for fieldIndex in fieldIndices:
        fieldValue = str(row[fieldIndex])
        print(fieldValue.ljust(FIELD_MAX_WIDTH), end=' ')

    print() # kiíratás végén sorvégjel kitétele


cur.close()
con.close()

Törölve
---- Kiíratás 11  ----

Lajos
57
Béla
53
Ildikó
54
Dani
22
Tomi
21
---- Kiíratás 2.  ----

Lajos is 57 years old.
Béla is 53 years old.
Ildikó is 54 years old.
Dani is 22 years old.
Tomi is 21 years old.
Most törlök  1 rekordot/sort.
---- Kiíratás 3.  ----

with column names: Lajos
with column names: ('Béla', 53, 'f796ac4ec7dd2fc3a28fa2377b2cee9f')
[('Ildikó', 54, '77a17c0633ae98c1f082cf2701739473'), ('Dani', 22, '0b2cc82e6a177b18faefd88581b8597d')]
---- Kiíratás 4.  ----

name_last            age                  name_md5             
------------------------------------------------------------------------------
Lajos                57                   b48b8adcf3de3785f09f9900af1a3aa6 
Béla                 53                   f796ac4ec7dd2fc3a28fa2377b2cee9f 
Ildikó               54                   77a17c0633ae98c1f082cf2701739473 
Dani                 22                   0b2cc82e6a177b18faefd88581b8597d 


## Táblák lekérdezése

In [0]:
DB_FILE = "/content/sample_data/flights_information.db"  # adatbázis neve
#DB_FILE = "/content/sample_data/mydb.db"  # adatbázis neve
con = sqlite3.connect(DB_FILE)  ## ha nem létezik létrehozza
cur = con.cursor()
# cur.execute("select * from airports where id <= 10; ")
# print(cur.fetchall())  ## Összes további kiíratása

print('---- Kiíratás 11  ----\n')
cur.execute("SELECT * FROM sqlite_master WHERE type='table'")
print('\n---- Tábla infó kiíratás 1.  ----\n')
for row in cur:
    for mezo in row:
        print(mezo)

print('\n---- Verzió kiíratás 2.  ----\n')
cur.execute('select sqlite_version();')
for row in cur:
    for mezo in row:
        print(mezo)

cur.close()
con.close()

---- Kiíratás 11  ----


---- Tábla infó kiíratás 1.  ----

table
airports
airports
2
CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
)
table
airlines
airlines
3
CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
)
table
routes
routes
4
CREATE TABLE routes (
[index] INTEGER,
  [airline] TEXT,
  [airline_id] TEXT,
  [source] TEXT,
  [source_id] TEXT,
  [dest] TEXT,
  [dest_id] TEXT,
  [codeshare] TEXT,
  [stops] TEXT,
  [equipment] TEXT
)

---- Verzió kiíratás 2.  ----

3.22.0


In [0]:
import sqlite3
DB_FILE = "/content/sample_data/mydb.db"  # adatbázis neve
con = sqlite3.connect(DB_FILE)  ## ha nem létezik létrehozza
cur = con.cursor()
newdb=0
con.create_function("md5", 1, md5sum)

cur = con.cursor()

if newdb == 1 :  ## Ha új adatbázis akkor a táblát léátrekell hozni
    cur.execute(""" create table people (
      name_last  varchar(20), 
      age        integer)  """)
 
    ## Rekordok beszúrása / INSERT
    cur.execute("insert into people (name_last, age) values ('Lajos',   57)")    
    cur.execute("insert into people (name_last, age) values ('Béla',     53)")   
    cur.execute("insert into people (name_last, age) values ('Ildikó',     54)")  
    con.commit()  ## Beírás véglegesítése

# cur.execute(" SELECT * FROM sqlite_master WHERE type='table' ")
cur.execute(" SELECT * FROM people WHERE 1=1 ")

for row in cur:
    for mezo in row:
        print(mezo)

cur.close()
con.close()

Lajos
57
Béla
53
Ildikó
54
Dani
22
Tomi
21
